In [1]:
import pandas as pd
from konlpy.tag import Okt; t = Okt()
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
train = pd.read_csv('./datas/train_ver1', index_col=[0])
train.tail()

,comments,bias_label,gender_label,hate_label,news_title,comment_pos,title_pos
7891,"힘내세요,응원합니다",none,False,none,"허지웅 허투루 넘길 말 없었다,솔직하게 드러냈던 속 사정","[('힘내세요', 'Verb'), (',', 'Punctuation'), ('응원'...","[('허지웅', 'Noun'), ('허투루', 'Noun'), ('넘길', 'Ver..."
7892,"힘내세요,삼가 고인의 명복을 빕니다",none,False,none,이혜경 오 캐롤 공연 중 남편 오정욱 부 고 오열 속 발인 종합,"[('힘내세요', 'Verb'), (',', 'Punctuation'), ('삼가'...","[('이혜경', 'Noun'), ('오', 'Noun'), ('캐롤', 'Noun'..."
7893,힘내세용 항상 응원합니닷,none,False,none,설경구 송윤아 아들과 즐거운 하루 전 엄마니까요,"[('힘내세용', 'Verb'), ('항상', 'Noun'), ('응원', 'Nou...","[('설경구', 'Noun'), ('송윤아', 'Noun'), ('아들', 'Nou..."
7894,"힘내 소연기로 답해요,나도 53살 인데 이런 일 저런 일 다 있더라구 요인격을 믿습...",none,False,none,SC 현장 연예인 인생 협박 유감 미소 잃은 최민수 보복운전 혐의 2차 공판 종합,"[('힘내', 'Verb'), ('소', 'Modifier'), ('연기', 'No...","[('SC', 'Alpha'), ('현장', 'Noun'), ('연예인', 'Nou..."
7895,힘들면 관뒀어야지 그게 현명한 거다,none,False,none,단독 스태프 사망 사고 서른 이지만 결국 오늘 촬영 취소,"[('힘들면', 'Adjective'), ('관', 'Noun'), ('뒀어야지',...","[('단독', 'Noun'), ('스태프', 'Noun'), ('사망', 'Noun..."


In [3]:
test = pd.read_csv('./datas/test.hate.no_label.csv')
test.tail()

,comments
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...
972,입에 손가릭이 10개 있으니 징그럽다
973,난 조보아 이뻐서 보는데 백종원 관심무


In [15]:
dev = pd.read_csv('./datas/dev.hate.csv')
dev.rename(columns={'label': 'hate_label'}, inplace=True)

In [4]:
# define for tokenizer parameter for tfidf
def t_tokenizer(text):
    tokens_ko = t.morphs(text)
    return tokens_ko

In [5]:
tfidf_vect = TfidfVectorizer(tokenizer=t_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train['comments'])
tfidf_matrix_train = tfidf_vect.transform(train['comments'])

## LGBMClassifier compare train and dev data

In [12]:
from lightgbm import LGBMClassifier
import time

start_time = time.time()
lgbm_clf = LGBMClassifier(n_estimators=400)
lgbm_clf.fit(tfidf_matrix_train, train['hate_label'])
print('fit time:', time.time() - start_time)

fit time: 14.255283117294312


In [13]:
from sklearn.metrics import f1_score, accuracy_score, recall_score

tfidf_matrix_test = tfidf_vect.transform(test['comments'])
preds = lgbm_clf.predict(tfidf_matrix_test)

## DecisionTreeClassifier compare train and dev data

In [10]:
from sklearn.tree import DecisionTreeClassifier

decision_clf = DecisionTreeClassifier(max_depth=2, random_state=13)
decision_clf.fit(tfidf_matrix_train, train['hate_label'])
print('fit time:', time.time() - start_time)

fit time: 8.65710735321045


In [11]:
preds = decision_clf.predict(tfidf_matrix_dev)
accuracy_score(dev['hate_label'], preds), f1_score(preds, dev['hate_label'], average='macro')

(0.4118895966029724, 0.33302547396349347)

## LogisticRegression compare train and dev data

In [16]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='liblinear', random_state=13)
lr.fit(tfidf_matrix_train, train['hate_label'])
print('fit time:', time.time() - start_time)

fit time: 97.49897718429565


In [17]:
preds = lr.predict(tfidf_matrix_test)
len(preds)

974

In [18]:
preds = pd.DataFrame(preds)
preds[preds[0] == 'none'] = 0

In [19]:
preds[preds[0] == 'offensive'] = 1
preds[preds[0] == 'hate'] = 2
preds

,0
0,0
1,0
2,0
3,0
4,1
...,...
969,0
970,1
971,0
972,2


In [20]:
preds['comments'] = test['comments']

In [21]:
preds = preds[['comments', 0]]
preds.rename(columns={0: 'label'}, inplace=True)
preds.tail()

,comments,label
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,1
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,0
972,입에 손가릭이 10개 있으니 징그럽다,2
973,난 조보아 이뻐서 보는데 백종원 관심무,0


In [35]:
preds.to_csv('./datas/NaiveBayeji_prediction.csv', index=False)

In [36]:
pd.read_csv('./datas/NaiveBayeji_prediction.csv')


,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,0
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,1
...,...,...
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,1
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,0
972,입에 손가릭이 10개 있으니 징그럽다,2


## GradientBoostingClassifier compare train and dev data

In [53]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(random_state=13)
gb_clf.fit(tfidf_matrix_train, train['hate_label'])
print('fit time:', time.time() - start_time)

fit time: 944.8058462142944


In [54]:
preds = gb_clf.predict(tfidf_matrix_dev)
accuracy_score(dev['hate_label'], preds), f1_score(preds, dev['hate_label'], average='macro')

(0.5095541401273885, 0.48810533169257236)

In [56]:
!pip install xgboost

## XGBClassifier compare train and dev data

In [12]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb_clf.fit(tfidf_matrix_train, train['hate_label'])
print('fit time:', time.time() - start_time)

[00:20:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time: 153.06296706199646


In [13]:
preds = xgb_clf.predict(tfidf_matrix_dev)
accuracy_score(dev['hate_label'], preds), f1_score(preds, dev['hate_label'], average='macro')

(0.5222929936305732, 0.5038136262817716)

In [59]:
from sklearn.model_selection import train_test_split

msg_train, msg_test, class_train, class_test = train_test_split(tfidf_matrix_train, train['hate_label'], test_size=0.2, stratify=train['hate_label'])

In [60]:
from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

models = []
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
models.append(('LogisticRegression', LogisticRegression()))
models.append(('LGBMClassifier', LGBMClassifier()))

In [61]:
from sklearn.metrics import f1_score, accuracy_score, recall_score

for name, model in models:
    clf = model
    clf.fit(msg_train, class_train)
    
    y_pred_tr = clf.predict(msg_train)
    y_pred_test = clf.predict(msg_test)
    
    print('Train Acc : ', accuracy_score(class_train, y_pred_tr))
    print('Test Acc : ', accuracy_score(class_test, y_pred_test))

Train Acc :  0.9988917036098797
Test Acc :  0.5360759493670886
Train Acc :  0.9988917036098797
Test Acc :  0.46392405063291137
Train Acc :  0.5582647245091831
Test Acc :  0.510759493670886
Train Acc :  0.652786573780874
Test Acc :  0.5259493670886076
Train Acc :  0.8530715642811906
Test Acc :  0.5430379746835443
Train Acc :  0.8096896770107663
Test Acc :  0.5316455696202531


In [62]:
train_score = []
test_score = []
names = []
f1score = []

for name, model in models:
    clf = model
    clf.fit(msg_train, class_train)
    
    y_pred_tr = clf.predict(msg_train)
    y_pred_test = clf.predict(msg_test)
    
    names.append(name)
    test_score.append(accuracy_score(class_test, y_pred_test))
    train_score.append(accuracy_score(class_train, y_pred_tr))
    f1score.append(f1_score(class_test, y_pred_test, average='macro'))

In [63]:
result = pd.DataFrame({'model name':names, 
                       'train score':train_score, 
                       'test score':test_score,
                       'f1 score': f1score}) 
result

,model name,train score,test score,f1 score
0,RandomForestClassifier,0.998892,0.532911,0.474092
1,DecisionTreeClassifier,0.998892,0.473418,0.441096
2,AdaBoostClassifier,0.558265,0.510759,0.444480
3,GradientBoostingClassifier,0.652787,0.524684,0.451602
4,LogisticRegression,0.853072,0.543038,0.507301
5,LGBMClassifier,0.809690,0.531646,0.498601
